## General imports

In [1]:
import os,sys,glob,shutil
import imageio
import numpy as np
import matplotlib.pyplot as plt
plt.rc('image',cmap='Greys')
import tifffile
%matplotlib inline

## ```ClearMap2``` imports

In [2]:
sys.path.append('/jukebox/witten/Chris/python/ClearMap2-master')
import ClearMap.IO.Workspace as wsp
import ClearMap.IO.IO as io
import ClearMap.ImageProcessing.Experts.Cells as cells
import ClearMap.Utils.HierarchicalDict as hdict

## Select dataset to analyze

In [ ]:
fpath = 'zimmerman_01/zimmerman_01-001/imaging_request_1/rawdata/resolution_3.6x/'

## Link and rename image files

In [ ]:
# src_dir = os.path.join('/jukebox/LightSheetData/lightserv/cz15',fpath)
# dst_dir = os.path.join('/jukebox/witten/Chris/data/clearmap2',fpath)

# src_488 = os.path.join(src_dir,'Ex_488_Em_0/corrected')
# dst_488 = os.path.join(dst_dir,'Ex_488_Em_0/corrected')
# os.makedirs(dst_488)
# src_files = sorted(glob.glob(src_488 + '/*tif'))
# for ii,src in enumerate(src_files):
#     dst_basename = 'Z'+f'{ii}'.zfill(4)+'.tif'
#     dst = os.path.join(dst_488,dst_basename)
#     os.symlink(src,dst)

# src_642 = os.path.join(src_dir,'Ex_642_Em_2/corrected')
# dst_642 = os.path.join(dst_dir,'Ex_642_Em_2/corrected')
# os.makedirs(dst_642)
# src_files = sorted(glob.glob(src_642 + '/*tif'))
# for ii,src in enumerate(src_files):
#     dst_basename = 'Z'+f'{ii}'.zfill(4)+'.tif'
#     dst = os.path.join(dst_642,dst_basename)
#     os.symlink(src,dst)

## Initialize ```ClearMap2``` workspace object

In [ ]:
directory = os.path.join('/jukebox/witten/Chris/data/clearmap2',fpath)
expression_auto = '/Ex_488_Em_0/corrected/Z<Z,4>.tif'
expression_raw = '/Ex_642_Em_2/corrected/Z<Z,4>.tif'
ws = wsp.Workspace('CellMap',directory=directory)
ws.update(raw=expression_raw)
ws.update(autofluorescence=expression_auto)
ws.debug = False
ws.info()

## Verify images were correctly loaded

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot()
z_plane = ws.source('raw')[:,:,2500]
ax.imshow(z_plane,vmin=0,vmax=200);

## Created stitched image volume

In [ ]:
# source = ws.source('raw');
# sink = ws.filename('stitched')
# io.convert(source,sink,verbose=True)

## Select image subvolume to analyze

In [ ]:
ws.debug = 'subvolume_0'
slicing = (slice(4000,4500),slice(2000,2500),slice(1450,1480))

ws.debug = 'subvolume_1'
slicing = (slice(1500,2000),slice(2000,2500),slice(1450,1480))

ws.create_debug('stitched',slicing=slicing)

In [ ]:
fig = plt.figure(figsize=(15,10))
ax=fig.add_subplot(111)
ax.imshow(np.amax(ws.source('stitched'),2),vmin=0,vmax=400);

## Run ```CellMap``` on image subvolume

In [ ]:
cell_detection_parameter = cells.default_cell_detection_parameter.copy()
cell_detection_parameter['iullumination_correction'] = None
cell_detection_parameter['background_correction']['shape'] = (15,15)
cell_detection_parameter['background_correction']['form'] = 'Disk'
cell_detection_parameter['background_correction']['save'] = False
cell_detection_parameter['equalization'] = None
cell_detection_parameter['dog_filter'] = None
cell_detection_parameter['maxima_detection']['h_max'] = None
cell_detection_parameter['maxima_detection']['shape'] = 10
cell_detection_parameter['maxima_detection']['threshold'] = None
cell_detection_parameter['maxima_detection']['valid'] = True
cell_detection_parameter['maxima_detection']['save'] = False
cell_detection_parameter['shape_detection']['threshold'] = 150
cell_detection_parameter['shape_detection']['save'] = False
cell_detection_parameter['intensity_detection']['method'] = 'max'
cell_detection_parameter['intensity_detection']['shape'] = 10
cell_detection_parameter['intensity_detection']['measure'] = ['source','background']
cell_detection_parameter['verbose'] = False
hdict.pprint(cell_detection_parameter)

In [ ]:
processing_parameter = cells.default_cell_detection_processing_parameter.copy()
processing_parameter.update(processes='serial',size_max=100,size_min=30,overlap=15,verbose=True)
cells.detect_cells(ws.filename('stitched'),ws.filename('cells',postfix='raw'),
                   cell_detection_parameter=cell_detection_parameter,
                   processing_parameter=processing_parameter)

## Plot distribution of detected cells

In [ ]:
source = ws.source('cells',postfix='raw')
fig = plt.figure(figsize=(12,10))
plt.figure(1)
plt.clf()
names = source.dtype.names
nx,ny = 2,3
for i, name in enumerate(names):
    plt.subplot(nx,ny,i+1)
    plt.hist(source[name])
    plt.title(name)
plt.tight_layout();

## Filter and visualize detected cells

In [ ]:
thresholds = {'background':None,'size':(5,None)}
cells.filter_cells(source=ws.filename('cells',postfix='raw'),
                   sink=ws.filename('cells',postfix='filtered'),
                   thresholds=thresholds);

In [ ]:
coordinates = np.hstack([ws.source('cells',postfix='filtered')[c][:,None] for c in 'xyz'])
coordinates = np.delete(coordinates,obj=2,axis=1)
xs = coordinates[:,0]
ys = coordinates[:,1]
fig,axes = plt.subplots(figsize=(15,10),nrows=1,ncols=2,sharex=True,sharey=True)
ax_tissue = axes[0]
ax_tissue.imshow(np.amax(ws.source('stitched'),2),vmin=0,vmax=400)
ax_both=axes[1]
ax_both.imshow(np.amax(ws.source('stitched'),2),vmin=0,vmax=400)
ax_both.scatter(ys,xs,s=50,facecolors='none',edgecolors='r')
plt.savefig(directory+'/'+ws.debug+'_cells.png');

In [ ]:
filenames = []
data = ws.source('stitched')
coordinates = np.hstack([ws.source('cells',postfix='filtered')[c][:,None] for c in 'xyz'])
xs = coordinates[:,0]
ys = coordinates[:,1]
zs = coordinates[:,2]
for i in range(0, 29):
    fig = plt.figure(figsize=(15,10))
    plt.imshow(data[:,:,i],vmin=0,vmax=200)
    zs1 = np.where((zs>(i-4))&(zs<(i+4)),True,False)
    ys1 = ys[zs1]
    xs1 = xs[zs1]
    plt.scatter(ys1,xs1,s=50,facecolors='none',edgecolors='r')
    filename = f'{i}.png'
    filenames.append(filename)
    plt.savefig(filename)
    plt.close()
with imageio.get_writer(directory+'/'+ws.debug+'_cells.gif',mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
for filename in set(filenames):
    os.remove(filename)
from IPython.display import Image
Image(filename=directory+'/'+ws.debug+'_cells.gif')